<a href="https://www.kaggle.com/code/abhayratti/cnn-with-pytorch?scriptVersionId=111696210" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [7]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import pandas as pd

In [6]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [16]:
all_transforms = transforms.Compose([transforms.Resize((32,32)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], 
                                                        std =[0.2023, 0.1994, 0.2010])\
                                    ])

train_dataset = torchvision.datasets.CIFAR10(root = "./data",
                                            train = True,
                                            transform = all_transforms,
                                            download = True)

test_dataset = torchvision.datasets.CIFAR10(root = "./data",
                                           train = False,
                                           transform = all_transforms,
                                           download = True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
        
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out

In [18]:
model = ConvNeuralNet(num_classes)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

total_step = len(train_loader)

In [19]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 1.7742
Epoch [2/20], Loss: 1.4596
Epoch [3/20], Loss: 0.8846
Epoch [4/20], Loss: 1.0588
Epoch [5/20], Loss: 1.1241
Epoch [6/20], Loss: 1.0226
Epoch [7/20], Loss: 0.8822
Epoch [8/20], Loss: 1.0253
Epoch [9/20], Loss: 1.1013
Epoch [10/20], Loss: 0.6995
Epoch [11/20], Loss: 0.9861
Epoch [12/20], Loss: 0.6343
Epoch [13/20], Loss: 0.5414
Epoch [14/20], Loss: 0.8114
Epoch [15/20], Loss: 0.6785
Epoch [16/20], Loss: 0.2784
Epoch [17/20], Loss: 0.9813
Epoch [18/20], Loss: 0.7905
Epoch [19/20], Loss: 0.6194
Epoch [20/20], Loss: 0.5405


In [20]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels, in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Acurracy on {} train images: {}%'.format(50000, 100 * correct / total))

Acurracy on 50000 train images: 82.804%
